In [ ]:
#import packages
import numpy as np
import matplotlib.pyplot as pl
from glob import glob
from astropy.io import fits

from scipy.stats import chisquare

from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation
import astropy.units as u
from astropy.table import Table

from uncertainties import ufloat
from uncertainties import unumpy as unp

from atmo_utilities import ccf, one_log_likelihood, log_likelihood_CCF, vacuum2air, log_likelihood_opt_beta

import time

from dtutils import psarr

from radiant import generate_atmospheric_model, get_wavelength_range

from create_model import create_model, instantiate_radtrans
import horus


### todo list
    - update HORUS fit for Doppler Shadow Removal
    - Check 1D slice of CCF plot - we can fit a gaussian curve or another curve to find parameters and also find how they vary over time.
        - wind speed
        - rotational line broadening
        - variation in center over time
    - check remaining species (mostly done!)
        - discrepancies between marshalls and your results may have to do with the previous vacuum2air conversion error (TiO)
    - scrutinize detections for winds

In [ ]:
#Importing Packages
from scipy.optimize import curve_fit

def gaussian(x, a, mu, sigma):

    '''
    Inputs:
    x: x values
    a: amplitude
    mu: mean
    sigma: standard deviation

    Output:
    Gaussian function
    '''
    return a * np.exp(-(x - mu)**2 / (2 * sigma**2))

# Fitting a Guassian to the 1D slice during transit

amps = []
amps_err = []
centers = []
centers_err = []
sigmas = []
sigmas_err = []

phase_slices = []

phase = 0.0

for i in range(len(orbital_phase)):
    current_slice = cross_cor[i, :]
    phase_slices.append(current_slice)
    popt, pcov = curve_fit(gaussian, drv, current_slice, p0=[1, 0, 1])

    amps.append(popt[0])
    centers.append(popt[1])
    sigmas.append(popt[2])

    # Storing errors (standard deviations)
    amps_err.append(np.sqrt(pcov[0, 0]))
    centers_err.append(np.sqrt(pcov[1, 1]))
    sigmas_err.append(np.sqrt(pcov[2, 2]))

# Selecting a specific phase slice
selected_idx = np.argmin(np.abs(orbital_phase - phase))
selected_slice = phase_slices[selected_idx]

# Fitting a Gaussian to the selected slice
popt_selected = [amps[selected_idx], centers[selected_idx], sigmas[selected_idx]]

# Plotting the fit parameters for the phase slice
pl.plot(drv, selected_slice, 'o', label='data')
pl.plot(drv, gaussian(drv, *popt_selected), 'r-', label='fit')
pl.legend()

pl.show()

breakpoint()